In [ ]:
%matplotlib inline

In [1]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import json
import operator 
from collections import Counter
from nltk.corpus import stopwords
import string
import nltk
import pandas as pd
from nltk import bigrams 
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
x = pd.DataFrame(count_all.most_common(20))
x

In [ ]:
# n_docs is the total n. of tweets
n_docs = len(listtweets)
p_t = {}
p_t_com = defaultdict(lambda : defaultdict(int))
 
for term, n in count_all.items():
    p_t[term] = n / n_docs
    for t2 in listtweets:
        p_t_com[term][t2] = listtweets[term][t2] / n_docs

In [ ]:
positive_vocab = [
    'good', 'nice', 'great', 'awesome', 'outstanding',
    'fantastic', 'terrific', ':)', ':-)', 'like', 'love',
    # shall we also include game-specific terms?
    # 'triumph', 'triumphal', 'triumphant', 'victory', etc.
]
negative_vocab = [
    'bad', 'terrible', 'crap', 'useless', 'hate', ':(', ':-(',
    # 'defeat', etc.
]

In [ ]:
from os import path
from wordcloud import WordCloud

#d = path.dirname(__file__)

# Read the whole text.
text = open( 'bitcoin.json').read()

# Generate a word cloud image
wordcloud = WordCloud().generate(text)

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate(text)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.savefig('foo.png')

In [ ]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
 
consumer_key = 'nYxXmA9B7MmpgRdQBzg9LZxCp'
consumer_secret = 'cE5yAit3vEAIyQIyM69uGuSMAxjh9y8cPWXu7lyR8HF0EB02D4'
access_token = '45810954-CGIPoVv7cSkYLG5XsXFXLv2B4p5yfLfg3uF7EFUcc'
access_secret = '4SzUyUbAbTePDOjet7cw6AAOWwV41Wy9mZzdJV6TCpgYT'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)
listy = []

 
class MyListener(StreamListener):
    def on_status(self, status):
        listy.append(status)
        print_ratios(listy)
        #print(status.created_at,status.text)
        

class MySaver(StreamListener): 
    def on_status(self, data):
        try:
            with open('bitcoin.json', 'a') as f:
                f.write(data.text)
                return True
        except BaseException as e:
            print(str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#bitcoin'])

1
Positive: 0.0 % Negative: 0.0 % Neutral:100.0%
2
Positive: 0.0 % Negative: 0.0 % Neutral:100.0%
3
Positive: 0.0 % Negative: 0.0 % Neutral:100.0%
4
Positive: 0.0 % Negative: 0.0 % Neutral:100.0%
5
Positive: 0.0 % Negative: 20.0 % Neutral:80.0%
6
Positive: 0.0 % Negative: 16.666666666666668 % Neutral:83.33333333333333%
7
Positive: 0.0 % Negative: 14.285714285714286 % Neutral:85.71428571428571%
8
Positive: 0.0 % Negative: 12.5 % Neutral:87.5%
9
Positive: 0.0 % Negative: 11.11111111111111 % Neutral:88.88888888888889%
10
Positive: 10.0 % Negative: 10.0 % Neutral:80.0%
11
Positive: 18.181818181818183 % Negative: 9.090909090909092 % Neutral:72.72727272727273%
12
Positive: 25.0 % Negative: 8.333333333333334 % Neutral:66.66666666666667%
13
Positive: 23.076923076923077 % Negative: 7.6923076923076925 % Neutral:69.23076923076923%
14
Positive: 28.571428571428573 % Negative: 7.142857142857143 % Neutral:64.28571428571429%
15
Positive: 33.333333333333336 % Negative: 6.666666666666667 % Neutral:60.0%

In [3]:
len(listy)

16

In [4]:
def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet))
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [5]:
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [13]:
def get_tweet(listy):
    tweets = []
    for tweet in listy:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
    
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = get_tweet_sentiment(tweet.text)
    
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
    return(tweets)

In [14]:
tweets = get_tweet(listy)

In [15]:
tweets

[{'sentiment': 'neutral',
  'text': 'The Public Sector is Embracing #bitcoin and the #blockchain https://t.co/MtjVV2dlRt https://t.co/hol0SeAiwt'},
 {'sentiment': 'positive',
  'text': '#survivor #Trump #bitcoin #Shakira #ParisHilton #gossip #Scandal #news #hot #summer #paparazzi Mom of 15-Year-Old M https://t.co/obg1IPYUyT'},
 {'sentiment': 'positive',
  'text': '#survivor #Trump #bitcoin #Shakira #ParisHilton #gossip #Scandal #news #hot #summer #paparazzi Mom of 15-Year-Old M https://t.co/udLRqsqLda'},
 {'sentiment': 'positive',
  'text': 'RT @silvercoin_io: Read our White Paper to find out exactly how SilverCoin works for you!\nhttps://t.co/HbCtCoDMSS\n\n$SVC #whitepaper… '},
 {'sentiment': 'positive',
  'text': 'RT @ipfconline1: The Blockchain Is "Just" a Powerful Distributed Ledger Technology [Infographic]\n#Blockchain #Banking #Fintech… '},
 {'sentiment': 'neutral',
  'text': 'RT @Cointelegraph: #Bitcoin Price Will Hit $250,000 by 2020 If Seven-Year Trend Continues\n[in case you 

In [28]:
def print_ratios(listy):
    tweets = get_tweet(listy)
    print(len(tweets))
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    print("Positive: {} %".format(100*len(ptweets)/len(tweets)),
      "Negative: {} %".format(100*len(ntweets)/len(tweets)),
      "Neutral:{}%".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets))
     )


In [27]:
print("Positive: {} %".format(100*len(ptweets)/len(tweets)),
      "Negative: {} %".format(100*len(ntweets)/len(tweets)),
      "Neutral:{}%".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets))
     )
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
print("Neutral tweets percentage:{}%".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))

Positive: 25.0 % Negative: 0.0 % Neutral:75.0%
Negative tweets percentage: 0.0 %
Neutral tweets percentage:75.0%
